<a href="https://colab.research.google.com/github/Kanade-nya/vits-japanese-remake/blob/main/%E2%80%9CVits_ipynb%E2%80%9D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/innnky/vits-japanese.git

Cloning into 'vits-japanese'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 96 (delta 18), reused 9 (delta 9), pack-reused 43
Unpacking objects: 100% (96/96), done.


In [ ]:
!pip install pypinyin Cython==0.29.21 librosa==0.8.0 matplotlib==3.3.1 numpy==1.18.5 phonemizer==2.2.1 scipy==1.5.2 Unidecode==1.1.1 >log.log
!sudo apt-get install espeak -y >log.log
!pip install gdown >log.log 
!pip install pyopenjtalk janome > log.log
!pip install cloud-tpu-client > log.log

In [ ]:
%cd /content/vits-japanese/monotonic_align
!python setup.py build_ext --inplace 

/content/vits-japanese/monotonic_align
running build_ext


In [ ]:
%cd /content/vits-japanese

import logging

numba_logger = logging.getLogger('numba')
numba_logger.setLevel(logging.WARNING)

import librosa

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text.cleaners import japanese_phrase_cleaners
from text import cleaned_text_to_sequence
from pypinyin import lazy_pinyin, Style

from scipy.io.wavfile import write

def get_text(text, hps):
    text_norm = cleaned_text_to_sequence(text)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm
# hps_ms = utils.get_hparams_from_file("./configs/vctk_base.json")
hps = utils.get_hparams_from_file("./configs/nan.json")
# net_g_ms = SynthesizerTrn(
#     len(symbols),
#     hps_ms.data.filter_length // 2 + 1,
#     hps_ms.train.segment_size // hps.data.hop_length,
#     n_speakers=hps_ms.data.n_speakers,
#     **hps_ms.model)

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model)
_ = net_g.eval()

def tts(text):
  stn_tst = get_text(text, hps)
  with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.float().numpy()
  ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))
def jtts(text):
  stn_tst = get_text(japanese_phrase_cleaners(text), hps)
  with torch.no_grad():
    x_tst = stn_tst.unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)])
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.float().numpy()
  ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate))

/content/vits-japanese


In [ ]:
!gdown "https://drive.google.com/u/0/uc?id=169Hbf3Qs4ahQ8EFRZCQtlGtVvp2f1Dd0&export=download" 

Downloading...
From: https://drive.google.com/u/0/uc?id=169Hbf3Qs4ahQ8EFRZCQtlGtVvp2f1Dd0&export=download
To: /content/vits-japanese/G_84000.pth
100% 436M/436M [00:01<00:00, 247MB/s]


In [ ]:
_ = utils.load_checkpoint("/content/vits-japanese/G_84000.pth", net_g, None)

In [ ]:
tts("nanika acltara itsudemo hanashIte kudasai. gakuiNno kotojanaku, shijini kaNsuru kotodemo nanidemo.")
tts("soodesUyo. jitsuwa watashitachiga nyuugakU suru maekara aclta bukatsuna NdesU. imawa buiNga inakUte, shozoku shIteru nowa watashI hitoridesUga.")
tts("nanode saikiNwa, gakUseekaino hookarawa keclkoo tsUtsukaretete...... katsudoomo, haclpyoonadoo iyokutekini shIte iru wakejaarimaseNkarane.")
tts("ichioo. shiclte haimasUkedo, watashini dekiru nowa tarocltoguraidesU. akumade shumiteedodesUkara.")
tts("a, hai. suguni ikimasU.")
tts("toshoshItsuwa...... moo daremo inai NdesUka?")

In [ ]:
jtts("吾輩は猫である。名前はまだない")
jtts("試験勉強頑張ってくださいね")
jtts("私の処女をもらってください")

Downloading: "https://github.com/r9y9/open_jtalk/releases/download/v1.11.1/open_jtalk_dic_utf_8-1.11.tar.gz"


dic.tar.gz: 0.00B [00:00, ?B/s]

Extracting tar file /usr/local/lib/python3.7/dist-packages/pyopenjtalk/dic.tar.gz


In [ ]:
%cd /content/vits-japanese

/content/vits-japanese


In [ ]:
!python preprocess.py --text_index 1 --filelists filelists/mzk_train.txt filelists/mzk_val.txt --text_cleaners japanese_phrase_cleaners

python3: can't open file 'preprocess.py': [Errno 2] No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/') #切换目录，这个是网盘的根目录
os.getcwd()#获取当前目录
if not os.path.exists('data'):
        os.makedirs('data')   #根目录下如果没有data文件夹，则创建data文件夹
 
os.chdir('data') #切换到data目录
os.chdir('..') #切换到上级目录

In [14]:
!ln -s /content/drive/MyDrive/data/wav/ /content/vits-japanese/

ln: failed to create symbolic link '/content/vits-japanese/wav': File exists


In [17]:
!python train.py -c configs/mzk_base.json -m mzk_base.json

INFO:mzk_base.json:{'train': {'log_interval': 50, 'eval_interval': 500, 'seed': 1234, 'epochs': 1000, 'learning_rate': 0.0002, 'betas': [0.8, 0.99], 'eps': 1e-09, 'batch_size': 32, 'fp16_run': True, 'lr_decay': 0.999875, 'segment_size': 8192, 'init_lr_ratio': 1, 'warmup_epochs': 0, 'c_mel': 45, 'c_kl': 1.0}, 'data': {'training_files': 'filelists/mzk_train.txt.cleaned', 'validation_files': 'filelists/mzk_val.txt.cleaned', 'text_cleaners': ['japanese_phrase_cleaners'], 'max_wav_value': 32768.0, 'sampling_rate': 22050, 'filter_length': 1024, 'hop_length': 256, 'win_length': 1024, 'n_mel_channels': 80, 'mel_fmin': 0.0, 'mel_fmax': None, 'add_blank': True, 'n_speakers': 0, 'cleaned_text': True}, 'model': {'inter_channels': 192, 'hidden_channels': 192, 'filter_channels': 768, 'n_heads': 2, 'n_layers': 6, 'kernel_size': 3, 'p_dropout': 0.1, 'resblock': '1', 'resblock_kernel_sizes': [3, 7, 11], 'resblock_dilation_sizes': [[1, 3, 5], [1, 3, 5], [1, 3, 5]], 'upsample_rates': [8, 8, 2, 2], 'upsam